In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = (
    SparkSession
    .builder
    .appName("solar-panel-tables-iceberg")
    .master("spark://2858efdf2609:7077")
    .config("spark.executor.cores", 1)
    .config("spark.cores.max", 6)
    .config("spark.executor.memory", "512M")
    .getOrCreate()
)

24/12/30 15:33:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
spark

# Read Weather CSV data

In [8]:
# Read Weather CSV data
from pyspark.sql import functions as F

_schema = "timestamp timestamp, solar_intensity float, temp float"

weather_df = spark.read.format("csv").schema(_schema).option("header", True)\
                   .load("/home/iceberg/warehouse/weather_history_splitted_resampled/2013-01-01.csv")\
                   .withColumn("15_min_interval", F.floor((F.hour(F.col("timestamp"))*60 + F.minute(F.col("timestamp")) - 60) / 15))                                                                                                 

In [9]:
weather_df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- solar_intensity: float (nullable = true)
 |-- temp: float (nullable = true)
 |-- 15_min_interval: long (nullable = true)



# Optimize Number of Partitions and Shuffle

In [10]:
# Check Spark Shuffle Partition setting

print("# shiffle: ", spark.conf.get("spark.sql.shuffle.partitions"))
print("# partitions: ", weather_df.rdd.getNumPartitions())

# shiffle:  200
# partitions:  6


In [11]:
spark.sparkContext.defaultParallelism

6

In [12]:
spark.conf.set("spark.sql.shuffle.partitions", 92)
weather_partitioned_df = weather_df.repartition(92, F.col('15_min_interval'))

# Craete solar panel tables

In [13]:
%%sql

CREATE DATABASE IF NOT EXISTS SolarX_Raw_Transactions

24/12/30 15:33:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [14]:
%%sql

DROP TABLE IF EXISTS SolarX_Raw_Transactions.solar_panel

++
||
++
++

In [15]:
%%sql
    
DROP TABLE IF EXISTS SolarX_Raw_Transactions.solar_panel_readings

++
||
++
++

In [16]:
%%sql
    
CREATE TABLE SolarX_Raw_Transactions.solar_panel(
    id INT,
    name VARCHAR(25) NOT NULL,
    capacity_kwh FLOAT NOT NULL,
    intensity_power_rating FLOAT NOT NULL,
    temperature_power_rating FLOAT NOT NULL
)
USING iceberg

++
||
++
++

In [17]:
%%sql

CREATE TABLE SolarX_Raw_Transactions.solar_panel_readings(
    timestamp TIMESTAMP NOT NULL,
    15_minutes_interval INT NOT NULL,
    panel_id INT NOT NULL,
    generated_power_amount FLOAT NOT NULL
)
USING iceberg
PARTITIONED BY (DAY(timestamp), 15_minutes_interval);

++
||
++
++

In [18]:
%%sql

INSERT INTO SolarX_Raw_Transactions.solar_panel (id, name, capacity_kwh, intensity_power_rating, temperature_power_rating) VALUES
(1, 'roof panel', 10, 1000, 25),
(2, 'pole panel', 12.5, 1300, 25),
(3, 'flush panel', 14, 1500, 25);

++
||
++
++

In [19]:
%%sql

SELECT * from SolarX_Raw_Transactions.solar_panel;

id,name,capacity_kwh,intensity_power_rating,temperature_power_rating
1,roof panel,10.0,1000.0,25.0
2,pole panel,12.5,1300.0,25.0
3,flush panel,14.0,1500.0,25.0


In [20]:
%%sql

SELECT * from SolarX_Raw_Transactions.solar_panel_readings;

timestamp,15_minutes_interval,panel_id,generated_power_amount


In [21]:
%%sql

SELECT * FROM SolarX_Raw_Transactions.solar_panel.files

content,file_path,file_format,spec_id,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
0,s3://warehouse/SolarX_Raw_Transactions/solar_panel/data/00000-0-bdd594c0-69e2-4a75-b208-523644ff0166-0-00001.parquet,PARQUET,0,1,1450,"{1: 36, 2: 46, 3: 36, 4: 36, 5: 36}","{1: 1, 2: 1, 3: 1, 4: 1, 5: 1}","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0}","{4: 0, 5: 0, 3: 0}","{1: bytearray(b'\x01\x00\x00\x00'), 2: bytearray(b'roof panel'), 3: bytearray(b'\x00\x00 A'), 4: bytearray(b'\x00\x00zD'), 5: bytearray(b'\x00\x00\xc8A')}","{1: bytearray(b'\x01\x00\x00\x00'), 2: bytearray(b'roof panel'), 3: bytearray(b'\x00\x00 A'), 4: bytearray(b'\x00\x00zD'), 5: bytearray(b'\x00\x00\xc8A')}",None,[4],None,0,"Row(capacity_kwh=Row(column_size=36, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=10.0, upper_bound=10.0), id=Row(column_size=36, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=1), intensity_power_rating=Row(column_size=36, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=1000.0, upper_bound=1000.0), name=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=None, lower_bound='roof panel', upper_bound='roof panel'), temperature_power_rating=Row(column_size=36, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=25.0, upper_bound=25.0))"
0,s3://warehouse/SolarX_Raw_Transactions/solar_panel/data/00001-1-bdd594c0-69e2-4a75-b208-523644ff0166-0-00001.parquet,PARQUET,0,1,1450,"{1: 36, 2: 46, 3: 36, 4: 36, 5: 36}","{1: 1, 2: 1, 3: 1, 4: 1, 5: 1}","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0}","{4: 0, 5: 0, 3: 0}","{1: bytearray(b'\x02\x00\x00\x00'), 2: bytearray(b'pole panel'), 3: bytearray(b'\x00\x00HA'), 4: bytearray(b'\x00\x80\xa2D'), 5: bytearray(b'\x00\x00\xc8A')}","{1: bytearray(b'\x02\x00\x00\x00'), 2: bytearray(b'pole panel'), 3: bytearray(b'\x00\x00HA'), 4: bytearray(b'\x00\x80\xa2D'), 5: bytearray(b'\x00\x00\xc8A')}",None,[4],None,0,"Row(capacity_kwh=Row(column_size=36, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=12.5, upper_bound=12.5), id=Row(column_size=36, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=2, upper_bound=2), intensity_power_rating=Row(column_size=36, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=1300.0, upper_bound=1300.0), name=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=None, lower_bound='pole panel', upper_bound='pole panel'), temperature_power_rating=Row(column_size=36, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=25.0, upper_bound=25.0))"
0,s3://warehouse/SolarX_Raw_Transactions/solar_panel/data/00002-2-bdd594c0-69e2-4a75-b208-523644ff0166-0-00001.parquet,PARQUET,0,1,1456,"{1: 35, 2: 47, 3: 36, 4: 36, 5: 36}","{1: 1, 2: 1, 3: 1, 4: 1, 5: 1}","{1: 0, 2: 0, 3: 0, 4: 0, 5: 0}","{4: 0, 5: 0, 3: 0}","{1: bytearray(b'\x03\x00\x00\x00'), 2: bytearray(b'flush panel'), 3: bytearray(b'\x00\x00`A'), 4: bytearray(b'\x00\x80\xbbD'), 5: bytearray(b'\x00\x00\xc8A')}","{1: bytearray(b'\x03\x00\x00\x00'), 2: bytearray(b'flush panel'), 3: bytearray(b'\x00\x00`A'), 4: bytearray(b'\x00\x80\xbbD'), 5: bytearray(b'\x00\x00\xc8A')}",None,[4],None,0,"Row(capacity_kwh=Row(column_size=36, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=14.0, upper_bound=14.0), id=Row(column_size=35, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=3, upper_bound=3), intensity_power_rating=Row(column_size=36, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=1500.0, upper_bound=1500.0), name=Row(column_size=47, value_count=1, null_value_count=0, nan_value_count=None, lower_bound='flush panel', upper_bound='flush panel'), temperature_power_rating=Row(column_size=36, value_count=1, null_value_count=0, nan_value_count=0, lower_bound=25.0, upper_bound=25.0))"


In [22]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|         demo|
|spark_catalog|
+-------------+



In [23]:
spark.conf.get("spark.sql.catalog.demo")

'org.apache.iceberg.spark.SparkCatalog'

In [24]:
spark.sql("SHOW DATABASES IN demo").show()

+--------------------+
|           namespace|
+--------------------+
|SolarX_Raw_Transa...|
+--------------------+



# Calc Solar Power

In [25]:
solar_panele_df = spark.read.format("iceberg").load("demo.SolarX_Raw_Transactions.solar_panel")

In [26]:
solar_panel_rating = solar_panele_df.filter(solar_panele_df.id == 1) \
                                    .select("capacity_kwh", "intensity_power_rating", "temperature_power_rating") \
                                    .first()
solar_panel_rating

Row(capacity_kwh=10.0, intensity_power_rating=1000.0, temperature_power_rating=25.0)

In [27]:
def calc_solar_readings(panel_id, weather_partitioned_df):
    # Loading solar_panel table
    solar_panele_df = spark.read.format("iceberg").load("SolarX_Raw_Transactions.solar_panel")

    # Quering panel power ratings
    solar_panel_rating = solar_panele_df.filter(solar_panele_df.id == panel_id) \
                                    .select("capacity_kwh", "intensity_power_rating", "temperature_power_rating") \
                                    .first()

    solar_panel_rating_kwh       = solar_panel_rating["capacity_kwh"]
    solar_intensity_power_rating = solar_panel_rating["intensity_power_rating"]
    temp_power_rating            = solar_panel_rating["temperature_power_rating"]
    
    solar_panel_rating_w_5ms = solar_panel_rating_kwh*1000/(60*60*1000/5)


    # Calculating the power value into a new df
    solar_panel_readings_df = weather_partitioned_df.withColumn('current_generation_watt', solar_panel_rating_w_5ms \
                * (1 /(1 - (temp_power_rating - F.col("temp"))/(temp_power_rating))) \
                * (1 - (solar_intensity_power_rating - F.col("solar_intensity"))/solar_intensity_power_rating) \
                                                           ).drop("solar_intensity", "temp")

    return solar_panel_readings_df

## Inserting solar_panel_readings 1 data into iceberg

In [28]:
panel_id = 1
solar_panel_readings_df1 = calc_solar_readings(panel_id, weather_partitioned_df)
solar_panel_readings_df1.printSchema()
solar_panel_readings_df1.createOrReplaceTempView("temp_view_1")

root
 |-- timestamp: timestamp (nullable = true)
 |-- 15_min_interval: long (nullable = true)
 |-- current_generation_watt: double (nullable = true)



In [29]:
%%sql
    
INSERT INTO SolarX_Raw_Transactions.solar_panel_readings (timestamp, 15_minutes_interval, panel_id, generated_power_amount)
SELECT timestamp                  as timestamp,
       15_min_interval            as 15_minutes_interval,
       1                          as panel_id,
       current_generation_watt    as generated_power_amount
       
FROM temp_view_1

++
||
++
++

## Inserting solar_panel_readings 2 and 3 data into iceberg

In [30]:
panel_id = 2
solar_panel_readings_df2 = calc_solar_readings(panel_id, weather_partitioned_df)
solar_panel_readings_df2.createOrReplaceTempView("temp_view_2")

In [31]:
%%sql
    
INSERT INTO SolarX_Raw_Transactions.solar_panel_readings (timestamp, 15_minutes_interval, panel_id, generated_power_amount)
SELECT timestamp                  as timestamp,
       15_min_interval            as 15_minutes_interval,
       2                          as panel_id,
       current_generation_watt    as generated_power_amount
       
FROM temp_view_2

++
||
++
++

In [32]:
panel_id = 3
solar_panel_readings_df3 = calc_solar_readings(panel_id, weather_partitioned_df)
solar_panel_readings_df3.createOrReplaceTempView("temp_view_3")

In [33]:
%%sql
    
INSERT INTO SolarX_Raw_Transactions.solar_panel_readings (timestamp, 15_minutes_interval, panel_id, generated_power_amount)
SELECT timestamp                  as timestamp,
       15_min_interval            as 15_minutes_interval,
       3                          as panel_id,
       current_generation_watt    as generated_power_amount
       
FROM temp_view_3

++
||
++
++

## Some analysis

In [272]:
%%sql

SELECT * from SolarX_Raw_Transactions.solar_panel_readings limit 10;

timestamp,15_minutes_interval,panel_id,generated_power_amount
2013-01-01 19:00:00,72,2,0.0
2013-01-01 19:00:00.005000,72,2,0.0
2013-01-01 19:00:00.010000,72,2,0.0
2013-01-01 19:00:00.015000,72,2,0.0
2013-01-01 19:00:00.020000,72,2,0.0
2013-01-01 19:00:00.025000,72,2,0.0
2013-01-01 19:00:00.030000,72,2,0.0
2013-01-01 19:00:00.035000,72,2,0.0
2013-01-01 19:00:00.040000,72,2,0.0
2013-01-01 19:00:00.045000,72,2,0.0


In [273]:
%%sql

SELECT COUNT(DISTINCT(15_minutes_interval)) from SolarX_Raw_Transactions.solar_panel_readings limit 10;

count(DISTINCT 15_minutes_interval)
92


In [281]:
%%sql

SELECT panel_id, 15_minutes_interval, SUM(generated_power_amount) as power
FROM SolarX_Raw_Transactions.solar_panel_readings
GROUP BY panel_id, 15_minutes_interval
SORT BY power desc

panel_id,15_minutes_interval,power
1,34,3707.9093862380832
1,32,3695.013436084613
1,37,3583.8991967197508
1,33,3567.1697686873376
2,34,3565.2974866973236
2,32,3552.8975344654173
3,34,3460.715426978655
3,32,3448.6792070502415
1,39,3446.10297219269
2,37,3446.0569196464494


In [34]:
spark.stop()